In [1]:
import sys
import os

root = os.path.dirname(os.path.abspath("."))
sys.path.append(root)

import numpy as np
import pandas as pd

import predikit as pk

# Test Load DataFrame

In [2]:
from io import BytesIO
from pathlib import Path

def load_show(path_or_buf, extension=None, n=3, label=None, **kwargs):
    df = pk.DataFrameParser(path_or_buf=path_or_buf, extension=extension, **kwargs)  # type: ignore
    if label:
        display(f"From {label}")
    display(df.head(n))


# from a Buffered Input Stream
f = BytesIO(b"a,b,c\n1,2,3\n4,5,6\n7,8,9")
load_show(f, extension="csv", label="BytesIO")

# from a csv file
f = Path("./sample_data/airline_bumping.csv")
load_show(f, n=4, label="csv")

# from a pickle file
f = Path("./sample_data/stations.pickle")
load_show(f, n=5, label="pickle")

# from a parquet file
f = Path("./sample_data/Flights 1m.parquet")
load_show(f, n=3, label="parquet")

# from a dictionary
data = {
    "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    "Age": [np.nan, 23, 25, 50, np.nan, 50],
    "Credit": [np.nan, 400, np.nan, 50, 200, np.nan],
}
load_show(data, n=3, label="dict")

# from a list of dictionaries
data = [
    {"Name": "John", "Age": np.nan, "Credit": np.nan},
    {"Name": "Andrea", "Age": 23, "Credit": 400},
    {"Name": "Rose", "Age": 25, "Credit": np.nan},
    {"Name": "Linda", "Age": 50, "Credit": 50},
    {"Name": "Peter", "Age": np.nan, "Credit": 200},
    {"Name": "Meg", "Age": 50, "Credit": np.nan},
]
load_show(data, label="list of dicts")

# from a dictionary of Series
data = {
    'one': pd.Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two': pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])
}
load_show(data, label="dict of Series")

# from a 2d array
data = np.array([
    ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    [np.nan, 23, 25, 50, np.nan, 50],
    [np.nan, 400, np.nan, 50, 200, np.nan],
])

data = data.T
load_show(data, label="2d array", columns=["Name", "Age", "Credit"])


# ToDo => add tests for JSON and EXCEL

'From BytesIO'

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


'From csv'

,airline,year,nb_bumped,total_passengers
0,DELTA AIR LINES,2017,679,99796155
1,VIRGIN AMERICA,2017,165,6090029
2,JETBLUE AIRWAYS,2017,1475,27255038
3,UNITED AIRLINES,2017,2067,70030765


'From pickle'

...

,Name,Age,Credit
0,John,NaN,NaN
1,Andrea,23.0,400.0
2,Rose,25.0,NaN


'From dict of Series'

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0


'From 2d array'

,Name,Age,Credit
0,John,nan,nan
1,Andrea,23,400
2,Rose,25,nan


# Handling Missing Values

In [3]:
data = {
    "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    "Age": [np.nan, 23, 25, 50, np.nan, 50],
    "Credit": [np.nan, 400, np.nan, 50, 200, np.nan],
}

df = pk.DataFrameParser(data)
display(df)

mvp = pk.MissingValuesProcessor(
    strategy=pk.MissingValueStrategy.MEAN, add_indicator=True, verbose=True
)

mvp.fit_transform(df, cols=["Age", "Credit"])

df

,Name,Age,Credit
0,John,NaN,NaN
1,Andrea,23.0,400.0
2,Rose,25.0,NaN
3,Linda,50.0,50.0
4,Peter,NaN,200.0
5,Meg,50.0,NaN


2023-12-10 21:28:14,950 (data_cleansing.py:276) - WARNING: ! Attention Age - 33% Missing!
2023-12-10 21:28:14,954 (data_cleansing.py:276) - WARNING: ! Attention Credit - 50% Missing!


,Name,Age,Credit,Age_isNA,Credit_isNA
0,John,37.0,216.666667,1,1
1,Andrea,23.0,400.000000,0,0
2,Rose,25.0,216.666667,0,1
3,Linda,50.0,50.000000,0,0
4,Peter,37.0,200.000000,1,0
5,Meg,50.0,216.666667,0,1
